In [7]:
import os
from scipy.io import wavfile
import pyworld
import pysptk
import IPython
from IPython.display import Audio
import numpy as np
from nnmnkwii.preprocessing.alignment import DTWAligner
from nnmnkwii.metrics import melcd
import matplotlib.pyplot as plt
import sklearn
import tqdm

import numpy as np
import IPython
from IPython.display import Audio
import torch
import torch.nn as nn
import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [8]:
n_mfcc = 40
NAME =  ...
DATASET_PATH = f"../data_generation/{NAME}/recordings"
INPUT_DIR = "/joanna"
OUTPUT_DIR = "/human"

In [ ]:
inputs = []
outputs = []

fs = 22050
fftlen = pyworld.get_cheaptrick_fft_size(fs)
alpha = pysptk.util.mcepalpha(fs)

In [ ]:
for file in sorted(os.listdir(DATASET_PATH + INPUT_DIR)):
    path = os.path.join(DATASET_PATH + INPUT_DIR, file)
    fs, data = wavfile.read(path)
    inputs.append(data)

for file in sorted(os.listdir(DATASET_PATH + OUTPUT_DIR)):
    path = os.path.join(DATASET_PATH + OUTPUT_DIR, file)
    fs, data = wavfile.read(path)
    outputs.append(data)

IPython.display.display(Audio(inputs[3], rate = fs))
IPython.display.display(Audio(outputs[3], rate = fs))

In [ ]:
def preprocess(data):
    data = data.astype(np.float64)
    f0, sp, ap = pyworld.wav2world(data, fs)
    mcc = pysptk.sp2mc(sp, order=n_mfcc, alpha = alpha)
    return mcc, f0, ap

In [ ]:

preprocessed_inputs = []
preprocessed_outputs = []

for data in tqdm.tqdm(inputs, "INPUTS"):
    preprocessed_inputs.append(preprocess(data))
for data in tqdm.tqdm(outputs, "OUTPUTS"):
    preprocessed_outputs.append(preprocess(data))

In [ ]:
def decode(mcc, f0, ap):
    sp = pysptk.mc2sp(
            mcc.astype(np.float64), alpha=alpha, fftlen=fftlen)
    waveform = pyworld.synthesize(
            f0, sp, ap, fs)
    return waveform

In [ ]:
# Test preprocess and decode
mcc, f0, ap = preprocessed_inputs[3]
waveform = decode(mcc, f0, ap)
IPython.display.display(Audio(waveform, rate = fs))

mcc, f0, ap = preprocessed_outputs[3]
waveform = decode(mcc, f0, ap)
IPython.display.display(Audio(waveform, rate = fs))

In [ ]:
aligner = DTWAligner(verbose=0, dist=melcd)
def align(X, Y):
    X, Y = aligner.transform((np.array([X]), np.array([Y])))
    return X[0], Y[0]

aligned_inputs = []
aligned_outputs = []
for i in range(len(preprocessed_inputs)):
    mcc_input, mcc_output = preprocessed_inputs[i][0], preprocessed_outputs[i][0]
    aligned_input, aligned_output = align(mcc_input,mcc_output)
    aligned_inputs.append(aligned_input)
    aligned_outputs.append(aligned_output)

In [ ]:
plt.imshow(preprocessed_inputs[3][0], aspect = 'auto')
plt.show()
plt.imshow(preprocessed_outputs[3][0], aspect = 'auto')

In [ ]:
plt.imshow(aligned_inputs[3], aspect = 'auto')
plt.show()
plt.imshow(aligned_outputs[3], aspect = 'auto')

In [ ]:
X = np.vstack(aligned_inputs)
Y = np.vstack(aligned_outputs)

X = torch.tensor(X)
Y = torch.tensor(Y)

torch.save(X,f'X_{NAME}')
torch.save(Y,f'Y_{NAME}')